In [2]:
# import os


# class uitiles:
#     DataSet_DF_columns=["filename","speaker","language","length","gender","accent","datasetname"]

    
#     def Generate_File_Name(Language :str, Gender:str, I_Speaker:int, I_Clip:int,datasetname: str,Extention: str):
#         return Language + "-" + Gender + "-" + str(I_Speaker) + "-" + str(I_Clip) + "-" + datasetname + "." + Extention

#     def Save_To_CSV(PathToDF, df):
#         if os.path.exists(PathToDF):
#             df.to_csv(PathToDF, mode='a', header=False)
#         else :
#             df.to_csv(PathToDF)


In [3]:
import tarfile
import pandas as pd 
import os
import shutil
import librosa
from IPython.display import clear_output

class ReadVoxForg:
    """This Class aims to read tgz and unzip this files in folders within same language"""

    _Path ="../../DataSets/InititalVoxForge"

    def GetPath(self,Path):
        _path=Path

    def ReadFileNames(self):
        """Read The text file contains files name and language
        
        Return: 
        DataFrame with FileNames and Languages
        """
        files = pd.read_csv(self._Path+"/voxforge_urls.txt",index_col=False, header= None ,  names=["FileNames"])
        filesNames_Lang = files["FileNames"].str.split('/', expand = True)[[2,7]]
        filesNames_Lang.columns=["Language","FileName"]
        filesNames_Lang=filesNames_Lang[filesNames_Lang["Language"]!='ru'] #As ru is not within the desired languages
        return filesNames_Lang

    def ReadFiles(self):   
        """Extact files in sperated folders based on the language""" 
        files = self.ReadFileNames() 
        Languages=  files["Language"].unique()
        ExtractedFiles = 0
        FailedFiles = 0
        for Language in Languages:
            for filename in files.loc[files["Language"]==Language,"FileName"]:
                try:
                    tar = tarfile.open(f'{self._Path}/{filename}', "r:*")
                    subdir_and_files = [
                            tarinfo for tarinfo in tar.getmembers()
                            if tarinfo.name.endswith(".wav")
                        ]
                    tar.extractall(members=subdir_and_files,path=f'../../DataSets/ExalVoxForge/{Language}')
                    tar.close()
                    ExtractedFiles=ExtractedFiles+1        
                except:
                    FailedFiles =FailedFiles+1
                    continue
        print(f"Extracted Files : {ExtractedFiles}")
        print(f"Field Files : {FailedFiles}")
    

    def MoveToOneFile(self,LangFilesName,Path,PathToMove,DataSetName,SaveToDF,PathToDF):
        """
        Move Specific langauges files in on folder

        Args:
            LangFilesName(list): list of folder names that contain the wav files

            Path(str):Path to languages files

            PathToMove(str):Path to Move

            DataSetName(str): Name of Dataset To create new name for each file
        """
        for  LangFile in LangFilesName:
            FilesNames =os.listdir(f"{Path}/{LangFile}")
            MovedFiles = []

            for I_Speaker, FilesName in enumerate(FilesNames) :
                
                print(f"File  : {I_Speaker} / {len(FilesNames)} - {LangFile}")
                splitedFilesName = FilesName.split('-')
                Speaker_Name=""
                if len(splitedFilesName) >=2 :
                   Speaker_Name= splitedFilesName[0] + "-" + splitedFilesName[1]
                else:
                    Speaker_Name= splitedFilesName[0] 

                lang = LangFile
                gender = "n"
                accent = ""
                for index, WavFile in enumerate (os.listdir(f"{Path}/{LangFile}/{FilesName}/wav")):
                    NewName= uitiles.Generate_File_Name(lang, gender, I_Speaker, index, DataSetName,"wav")  

                    try :
                        lenth =  librosa.get_duration(filename=f"{Path}/{LangFile}/{FilesName}/wav/{WavFile}")

                        shutil.copy2(f"{Path}/{LangFile}/{FilesName}/wav/{WavFile}", f'{PathToMove}/{NewName}')

                        MovedFiles.append([NewName,Speaker_Name,lang,lenth,gender,accent,DataSetName])
                    except BaseException as e:
                        print(f"Error While copy file {FilesName} - {WavFile} - ")
                        continue

                clear_output(wait=True)
        
            NewDF = pd.DataFrame(MovedFiles,columns=uitiles.DataSet_DF_columns)           
            if SaveToDF:
                uitiles.Save_To_CSV(PathToDF,NewDF)

            




In [4]:
# s =ReadVoxForg()
# a= s.ReadFileNames()
# s.ReadFiles()

In [5]:
u =ReadVoxForg()

LangFilesName =  ['de', 'en', 'es','fr', 'it']
u.MoveToOneFile(LangFilesName,"../../DataSets/ExalVoxForge","../../DataSets/FinalDataSet/clips" ,"VF",True,"../../DataSets/FinalDataSet/FinalDataSet.csv" )

File  : 1057 / 1058 - it


In [6]:
len("an4".split("-"))

1

In [7]:
1>=2

False

In [8]:
gg = pd.read_csv("../../DataSets/FinalDataSet/FinalDataSet.csv")

C:\Users\mm\AppData\Local\Temp\ipykernel_11172\3034147273.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  gg = pd.read_csv("../../DataSets/FinalDataSet/FinalDataSet.csv")


In [9]:
gg

,Unnamed: 0,filename,speaker,language,length,gender,accent,datasetname
0,0,ar-n-0-0-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,3.060000,n,NaN,CV
1,1,ar-n-0-1-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,4.176000,n,NaN,CV
2,2,ar-n-0-2-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,2.628000,n,NaN,CV
3,3,ar-n-0-3-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,3.168000,n,NaN,CV
4,4,ar-n-0-4-CV.wav,fc3b87e39142b5fced5eb2422f0f5277bdf471c8865e9a...,ar,4.716000,n,NaN,CV
...,...,...,...,...,...,...,...,...
249077,10549,it-n-1057-5-VF.wav,zartocle-20080812,it,6.826688,n,NaN,VF
249078,10550,it-n-1057-6-VF.wav,zartocle-20080812,it,6.570688,n,NaN,VF
249079,10551,it-n-1057-7-VF.wav,zartocle-20080812,it,7.338687,n,NaN,VF
249080,10552,it-n-1057-8-VF.wav,zartocle-20080812,it,7.253375,n,NaN,VF


In [22]:
gg.loc[:,"language" ].unique()

array(['ar', 'pt', 'de', 'en', 'es', 'fr', 'it'], dtype=object)